In [1]:
import pandas as pd
from indralib.indra_time import IndraTime
import polars

from wiki_exporter_tools import read_wiki_data_as_pandas, remove_footnotes, date_clean, date_merge

In [2]:
url= "https://en.wikipedia.org/wiki/Timeline_of_Irish_history"
tables = read_wiki_data_as_pandas(url)

In [3]:
tables[0]

,Year,Date,Event
0,"c. 16,000 BC",NaN,"During the Last Glacial Maximum, Ireland is co..."
1,"c. 12,000 BC",NaN,A narrow channel forms between Prehistoric Ire...
2,"c. 10,000 BC",NaN,Carbon-dating on bear bones indicate the prese...
3,c. 8000 BC,NaN,Mesolithic hunter-gatherers migrate to Ireland
4,c. 6500 BC,NaN,Mesolithic hunter-gatherers occupy sites such ...
5,c. 4000 BC,NaN,Agriculture (including the keeping of livestoc...
6,c. 3500 BC,NaN,The Neolithic peoples of the Boyne Valley buil...


In [15]:
hist = []
last_date = "<unknown>"
last_name = "<unknown>"
for table_index in range(23):
    for i, row in enumerate(tables[table_index].iterrows()):
        year = row[1].iloc[0]
        md = row[1].iloc[1]
        if pd.isna(md):
            md = ""
        else:
            md = str(md)
        year = str(year)
        # print(year, md)
        date = date_merge(year, md)
        date, rem = date_clean(date)
        remarks = remove_footnotes(row[1].iloc[2])
        if rem != "":
            remarks = f"({rem}) {remarks}"
        # print(date, remarks)

        jd = IndraTime.string_time_to_julian(date)
        if len(jd) == 1:
            jd = (jd[0], jd[0])
        hist.append((jd, date, remarks))

entries = sorted(hist, key=lambda x: x[0])
print(f"| Date | Remarks |")
print( "| ---- | ------- |")
for entry in entries:
    _, date, remarks = entry
    print(f"| {date} | {remarks} |")

| Date | Remarks |
| ---- | ------- |
| 17949 BP | (c.) During the Last Glacial Maximum, Ireland is covered in ice sheets |
| 12000 BC | (c.) A narrow channel forms between Prehistoric Ireland and southwest Scotland |
| 10000 BC | (c.) Carbon-dating on bear bones indicate the presence of Paleolithic people in County Clare. |
| 8000 BC | (c.) Mesolithic hunter-gatherers migrate to Ireland |
| 6500 BC | (c.) Mesolithic hunter-gatherers occupy sites such as that at Mount Sandel in Ulster |
| 4000 BC | (c.) Agriculture (including the keeping of livestock, and crop farming) has its beginnings in Ireland, at sites such as the Céide Fields in Connacht |
| 3500 BC | (c.) The Neolithic peoples of the Boyne Valley built a complex of chamber tombs, standing stones and enclosures over a period of hundreds of years. (Newgrange itself is dated to 3300–2900 BC). |
| 2000 BC | (c.) Bronze Age technologies start to arrive in Ireland, including the moulding of Ballybeg-type flat axes, and the beginnings

In [6]:
        remarks = ""
        pre_rem = ""
        post_rem = ""
        na_rem = ""
        date_raw = row[1].iloc[2]
        name = row[1].iloc[1]
        if pd.isna(date_raw) or date_raw == "" or date_raw == "??":
            na_rem = f"<NA>, date from previous table entry {last_name} used! "
            date_raw = last_date
        else:
            last_date = date_raw
            last_name = name
        date_raw = remove_footnotes(date_raw)
        ind = date_raw.find("ka,")
        if ind != -1:
            post_rem = date_raw[ind+3:]
            date_raw = date_raw[:ind] + "ka"
        if date_raw.startswith("ca. "):
            pre_rem = date_raw[:4]
            date_raw = date_raw[4:]
        dri = date_raw.split('(')
        if len(dri) > 1:
            date_raw = dri[0]
            post_rem = '('+'('.join(dri[1:])
        ind = date_raw.find('?')
        if ind != -1:
            post_rem = date_raw[ind:]
            date_raw = date_raw[:ind]
            
        date_raw = date_raw.replace(',', '.').replace('-', '–')
        date_inter = date_raw
        dps = date_raw.split(' ')
        if len(dps)>1 and dps[1] == 'mya':
            if '–' in dps[0]:
                dpsi = dps[0].split('–')
                date_inter = f"{dpsi[0]} mya - {dpsi[1]} " + ' '.join(dps[1:])
                  
        # print(f"Date_raw: {date_raw}")
        
        date, remarks = date_clean(date_inter)
        remarks = pre_rem + remarks + post_rem + na_rem
        # print(f"cv d:{date}, r:{remarks}")
        spec = row[1].iloc[3]
        if pd.isna(spec):
            spec = ""
        spec = remove_footnotes(spec)
        disc_date = row[1].iloc[4]
        if pd.isna(disc_date):
            disc_date = ""
        disc_date = remove_footnotes(str(disc_date))
        locat = row[1].iloc[5]
        if pd.isna(locat):
            locat = ""
        locat = remove_footnotes(locat)
        disc = row[1].iloc[6]
        if pd.isna(disc):
            disc = ""
        disc = remove_footnotes(disc)
        jd = IndraTime.string_time_to_julian(date)
        if len(jd) == 1:
            jd = (jd[0], jd[0])
        hist.append((jd, date, remarks, name, spec, disc_date, locat, disc))

entries = sorted(hist, key=lambda x: x[0])
print(f"| Date | Remarks | Name | Species | Discovery date | Location | First description |")
print( "| ---- | ------- | ---- | ------- | -------------- | -------- | ---------- |")
for entry in entries:
    _, date, remarks, name, spec, disc_date, locat, disc = entry
    date = date.replace("1 BC", "0 BC")
    print(f"| {date} | {remarks} | {name} | {spec} | {disc_date} | {locat} | {disc} |")

IndentationError: unexpected indent (1415617082.py, line 1)

In [5]:
url= "https://en.wikipedia.org/wiki/Flyby_anomaly"
tables = read_wiki_data_as_pandas(url)
ptab = polars.from_pandas(tables[0])
print(ptab)


shape: (11, 13)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ CraftData ┆ Galileo I ┆ Galileo   ┆ NEAR      ┆ … ┆ Juno      ┆ Hayabusa2 ┆ OSIRIS-RE ┆ BepiColo │
│ ---       ┆ ---       ┆ II        ┆ ---       ┆   ┆ ---       ┆ ---       ┆ x[8]      ┆ mbo[9]   │
│ str       ┆ str       ┆ ---       ┆ str       ┆   ┆ str       ┆ str       ┆ ---       ┆ ---      │
│           ┆           ┆ str       ┆           ┆   ┆           ┆           ┆ str       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Date      ┆ 1990-12-0 ┆ 1992-12-0 ┆ 1998-01-2 ┆ … ┆ 2013-10-0 ┆ 2015-12-0 ┆ 2017-09-2 ┆ 2020-04- │
│           ┆ 8         ┆ 8         ┆ 3         ┆   ┆ 9         ┆ 3         ┆ 2         ┆ 10       │
│ Speed at  ┆ 8.949     ┆ 8.877     ┆ 6.851     ┆ … ┆ null      ┆ 4.7       ┆ null      ┆ null     │
│ infinity, ┆           ┆           ┆           ┆   ┆           ┆          

In [6]:

# Transpose the table, get column names from previous index
ptabt = ptab.transpose(include_header=True)
print(ptabt)

shape: (13, 12)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ column    ┆ column_0  ┆ column_1  ┆ column_2  ┆ … ┆ column_7  ┆ column_8  ┆ column_9  ┆ column_1 │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ 0        │
│ str       ┆ str       ┆ str       ┆ str       ┆   ┆ str       ┆ str       ┆ str       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ CraftData ┆ Date      ┆ Speed at  ┆ Speed at  ┆ … ┆ Deflectio ┆ Speed     ┆ Speed     ┆ Gained   │
│           ┆           ┆ infinity, ┆ perigee,  ┆   ┆ n angle,  ┆ increment ┆ increment ┆ energy,  │
│           ┆           ┆ km/s      ┆ km/s      ┆   ┆ degrees   ┆ at        ┆ at        ┆ J/kg     │
│           ┆           ┆           ┆           ┆   ┆           ┆ infinity,

In [7]:
for i, row in enumerate(ptabt.iter_rows(named=True)):
    print("|", end="")
    for key in row:
        print(f" {row[key]} |", end="")
    print()
    if i == 0:
        print("|", end="")
        for key in row:
            print(f" {'-'*len(row[key])} |", end="")
        print()

| CraftData | Date | Speed at infinity, km/s | Speed at perigee, km/s | Impact parameter, km | Minimal altitude, km | Spacecraft mass, kg | Trajectory inclination to equator, degrees | Deflection angle, degrees | Speed increment at infinity, mm/s | Speed increment at perigee, mm/s | Gained energy, J/kg |
| --------- | ---- | ----------------------- | ---------------------- | -------------------- | -------------------- | ------------------- | ------------------------------------------ | ------------------------- | --------------------------------- | -------------------------------- | ------------------- |
| Galileo I | 1990-12-08 | 8.949 | 13.738 | 11261 | 956 | 2497.1 | 142.9 | 47.46 | 3.92±0.08 | 2.560±0.050 | 35.1±0.7 |
| Galileo II | 1992-12-08 | 8.877 | 8.877 | None | 303 | 2223.0 | 138.9 | 51.1 | −4.60±1.00 | −9.200±0.600 | None |
| NEAR | 1998-01-23 | 6.851 | 12.739 | 12850 | 532 | 730.40 | 108.0 | 66.92 | 13.46±0.13 | 7.210±0.0700 | 92.2±0.9 |
| Cassini | 1999-08-18 | 16.01 | 19

In [8]:
tables[0]

,CraftData,Galileo I,Galileo II,NEAR,Cassini,Rosetta-I,MESSENGER,Rosetta-II,Rosetta-III,Juno,Hayabusa2,OSIRIS-REx[8],BepiColombo[9]
0,Date,1990-12-08,1992-12-08,1998-01-23,1999-08-18,2005-03-04,2005-08-02,2007-11-13,2009-11-13,2013-10-09,2015-12-03,2017-09-22,2020-04-10
1,"Speed at infinity, km/s",8.949,8.877,6.851,16.01,3.863,4.056,NaN,NaN,NaN,4.7,NaN,NaN
2,"Speed at perigee, km/s",13.738,8.877,12.739,19.03,10.517,10.389,12.49,13.34,14.93,10.3,8.5,NaN
3,"Impact parameter, km",11261,NaN,12850,8973,22680.49,22319,NaN,NaN,NaN,NaN,NaN,19064
4,"Minimal altitude, km",956,303,532,1172,1954,2336,5322,2483,561[10],3090[11],17237,12677
5,"Spacecraft mass, kg",2497.1,2223.0,730.40,4612.1,2895.2,1085.6,2895,2895,~2720,590,NaN,4000
6,"Trajectory inclination to equator, degrees",142.9,138.9,108.0,25.4,144.9,133.1,NaN,NaN,NaN,NaN,NaN,NaN
7,"Deflection angle, degrees",47.46,51.1,66.92,19.66,99.396,94.7,NaN,NaN,NaN,80,NaN,NaN
8,"Speed increment at infinity, mm/s",3.92±0.08,−4.60±1.00,13.46±0.13,−2±1,1.82±0.05,0.02±0.01,~0,~0,0±0.8[5],?,?,?
9,"Speed increment at perigee, mm/s",2.560±0.050,−9.200±0.600,7.210±0.0700,−1.700±0.9000,0.670±0.0200,0.008±0.004,~0.000±0.000,−0.004±0.044,NaN,?,?,?
